In [1]:
import cv2
import numpy as np

In [15]:
#imgRGB = cv2.imread("Screenshot_3.png", cv2.IMREAD_UNCHANGED)
#imgHSV = cv2.cvtColor(imgRGB, cv2.COLOR_BGR2HSV)

cap = cv2.VideoCapture('sabrePDIcortado.mp4')

width= int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height= int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter('sabreFinalPDI23.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 30, (width,height))

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break    
        
    frameHSV = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    #Segmentação da parte azul(cabo)
    tomClaro = np.array([80,100,100])
    tomEscuro = np.array([140,255,255])
    imgSegmentada = cv2.inRange(frameHSV,tomClaro, tomEscuro)

    #Dilatação
    kernel = np.ones((5,5), np.uint8)
    dilation = cv2.dilate(imgSegmentada, kernel, iterations = 2)

    #Filtro mediana
    imgTratada  = cv2.medianBlur(dilation, 19)

    #Aplicando GaussianBlur
    blur = cv2.GaussianBlur(imgTratada,(81,81),0)

    #Misturando imagens
    frame[:,:,0] = cv2.add(frame[:,:,0],imgTratada)
    frame[:,:,1] = cv2.add(frame[:,:,1],imgTratada)
    frame[:,:,2] = cv2.add(frame[:,:,2],blur)
    frame[:,:,2] = cv2.add(frame[:,:,2],imgTratada)
    
    out.write(frame)
    cv2.imshow("Frame", frame)
    
    if cv2.waitKey(1) == ord('q'):
        break
        
cap.release()
out.release()
cv2.destroyAllWindows()